In [16]:
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor

In [17]:
MODEL = "openai:gpt-5"

In [ ]:
history_agent = create_react_agent(
    model=MODEL,
    tools=[],
    name="history_agent",
    prompt="You are a history expert. You only answer questions about history.",
)
geography_agent = create_react_agent(
    model=MODEL,
    tools=[],
    name="geography_agent",
    prompt="You are a geography expert. You only answer questions about geography.",
)
maths_agent = create_react_agent(
    model=MODEL,
    tools=[],
    name="maths_agent",
    prompt="You are a maths expert. You only answer questions about maths.",
)
philosophy_agent = create_react_agent(
    model=MODEL,
    tools=[],
    name="philosophy_agent",
    prompt="You are a philosophy expert. You only answer questions about philosophy.",
)

In [19]:
supervisor = create_supervisor(
    agents=[
        history_agent,
        maths_agent,
        geography_agent,
        philosophy_agent,
    ],
    model=init_chat_model(MODEL),
    prompt="""
    You are a supervisor that routes student questions to the appropriate subject expert. 
    You manage a history agent, geography agent, maths agent, and philosophy agent. 
    Analyze the student's question and assign it to the correct expert based on the subject matter:
        - history_agent: For historical events, dates, historical figures
        - geography_agent: For locations, rivers, mountains, countries
        - maths_agent: For mathematics, calculations, algebra, geometry
        - philosophy_agent: For philosophical concepts, ethics, logic
    """,
).compile()

In [20]:
questions = [
    "When was Madrid founded?",
    "What is the capital of France and what river runs through it?",
    "What is 15% of 240?",
    "Tell me about the Battle of Waterloo",
    "What are the highest mountains in Asia?",
    "If I have a rectangle with length 8 and width 5, what is its area and perimeter?",
    "Who was Alexander the Great?",
    "What countries border Switzerland?",
    "Solve for x: 2x + 10 = 30",
]

In [21]:
for question in questions:
    result = supervisor.invoke(
        {
            "messages": [
                {"role": "user", "content": question},
            ]
        }
    )
    if result["messages"]:
        for message in result["messages"]:
            message.pretty_print()

================================ Human Message =================================

When was Madrid founded?
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_history_agent (call_I5XWmAMvKZgt0C0DYepwym30)
 Call ID: call_I5XWmAMvKZgt0C0DYepwym30
  Args:
================================= Tool Message =================================
Name: transfer_to_history_agent

Successfully transferred to history_agent
================================== Ai Message ==================================
Name: history_agent

Madrid originated in the mid‑9th century, traditionally dated to 852–853 CE, when the Umayyad emir Muhammad I of Córdoba founded the fortress of Mayrit (Magerit) on the Manzanares River. It later became Spain’s capital in 1561 under Philip II.
================================== Ai Message ==================================
Name: history_agent

Transferring back to supervisor
Tool Calls:
  transfer_back_to_supervis